In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=611b9e4e5567f736df66bdf007afbe1f7fd3128fbb4c3d3a0f4d7e8feb6101a2
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [ ]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.getOrCreate()


In [28]:
# read the data
ori = spark.read.text("ego-facebook.txt")

value = []

dfc = ori.collect()

for row in dfc:
  value.append((int(row[0].split(' ')[0]),int(row[0].split(' ')[1])))
  value.append((int(row[0].split(' ')[1]),int(row[0].split(' ')[0])))

# generate the dataframe
df = spark.createDataFrame(value, ['id_1', 'id_2'])

In [29]:
# self left join and select three columns represent user, mutual friend and second degree friend
df_join = df.alias("df1").join(df.alias("df2"),
                               col("df1.id_2") == col("df2.id_1"),"left").select(col("df1.id_1"),
                              col("df2.id_1").alias("join_1"), 
                              col("df2.id_2").alias("join_2"))

# drop the duplicate rows
df_1 = df_join.filter(df_join.id_1 != df_join.join_2)

In [30]:
# group by user and second degree friend and count the mutual friend
gf = df_1.groupby(['id_1','join_2']).agg(size(collect_set('join_1')).alias("mf_count"))

# set up the dataframe for keys who actually mutual friend
sub = gf.alias('gd').join(df.alias('od'), (gf["id_1"] == df["id_1"]) 
                              & (gf["join_2"] == df["id_2"])).select(col('gd.id_1'),col('gd.join_2'),col('gd.mf_count'))

# take subtraction
gf = gf.subtract(sub)

In [31]:
# sorting
window = Window.partitionBy("id_1").orderBy(col("mf_count").desc(),col('join_2').asc())
gf = gf.withColumn("second_friend",collect_list("join_2").over(window)).groupby("id_1").agg(max('second_friend').alias("recommendation"))
gf = gf.orderBy(col('id_1').asc())

In [32]:
check = [10,152,288,603,714,1525,2434,2681]

for row in gf.collect():
  if row.id_1 in check:
    print(row.id_1,"\t",row.recommendation[:10])

10 	 [2, 3, 4, 5, 6, 7, 8, 9, 11, 12]
152 	 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
288 	 [71, 1525, 69, 90, 217, 2348, 2351, 2352, 2354, 2356]
603 	 [1, 289, 290, 291, 292, 293, 294, 295, 296, 297]
714 	 [1, 712, 713, 715, 717, 718, 1525, 90, 217, 247]
1525 	 [288, 1, 710, 714, 603]
2434 	 [71, 288, 711, 716, 719, 720, 2348, 2351, 2352, 2354]
2681 	 [71, 288, 710, 711, 716, 719, 720, 721, 722, 2348]
